# **Food Delivery App Recommendation System:**
#  **- Geohash Data Exploratory Analysis**

# **0. Load libraries**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



#pip install pygeohash folium
import pygeohash as pgh
#!pip install geohash2
import geohash2
#!pip install folium
import folium


# **1. Data Processing: orders and vendors geohashes**

## 1.1 Drop unnecessary columns, rename columns

In [56]:
# Import orders data
file_path = '/home/alnd/code/Alanoudis/food-delivery-rec/data/updated_data/orders_sg25k.txt'
orders = pd.read_csv(file_path)
orders.head(10)

,Unnamed: 0,customer_id,geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day
0,0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,12:03:29,85 days
1,1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,12:03:29,85 days
2,2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,12:03:29,85 days
3,3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,12:03:29,85 days
4,4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,12:03:29,85 days
5,5,f374c8c54c,w21zt,1,21830106,f245bdf79350,4,19:03:03,61 days
6,6,f374c8c54c,w21zt,1,21830106,146127be77d4,4,19:03:03,61 days
7,7,f374c8c54c,w21zt,1,21830106,6c108c0fb2b9,4,19:03:03,61 days
8,8,f374c8c54c,w21zt,1,21830106,da422c7836e1,4,19:03:03,61 days
9,9,f374c8c54c,w21zt,1,21830106,01110c80a0fa,4,19:03:03,61 days


In [57]:
# Drop unnecessary Columns (we won't use them in the analysis)
geo_df = orders.drop(columns=['Unnamed: 0', 'product_id', 'day_of_week', 'order_time', 'order_day'])

In [58]:
geo_df.head()

,customer_id,geohash,order_id,vendor_id
0,1ba124d4e5,w21z7,0,212753d2
1,1ba124d4e5,w21z7,0,212753d2
2,1ba124d4e5,w21z7,0,212753d2
3,1ba124d4e5,w21z7,0,212753d2
4,1ba124d4e5,w21z7,0,212753d2


In [59]:
# There are 32 geohash areas in the orders data
geo_df['geohash'].unique().shape

(32,)

In [60]:
# Rename customer geohash to customer_geohash for clarity

geo_df = geo_df.rename(columns={'geohash': 'customer_geohash'})
geo_df.head()

,customer_id,customer_geohash,order_id,vendor_id
0,1ba124d4e5,w21z7,0,212753d2
1,1ba124d4e5,w21z7,0,212753d2
2,1ba124d4e5,w21z7,0,212753d2
3,1ba124d4e5,w21z7,0,212753d2
4,1ba124d4e5,w21z7,0,212753d2


In [61]:
# Import vendors/chain data
file_path = '/home/alnd/code/Alanoudis/food-delivery-rec/data/raw_data/vendors_sg.txt'
vendors = pd.read_csv(file_path)
vendors.head()

,Unnamed: 0,vendor_id,chain_id,geohash,primary_cuisine
0,0,b160c319,d2786168,w21z6,mexican
1,1,9c8f010e,d2786168,w21ze,mexican
2,4,03eb25e1,5055ab25,w21ze,bak kut teh
3,5,3613129a,8984acb6,w23b1,italian
4,7,0946c9e5,1e3a2913,w21z4,bak kut teh


In [62]:
geo_vendors = vendors.drop(columns=['Unnamed: 0','primary_cuisine'])

In [63]:
geo_vendors.head()

,vendor_id,chain_id,geohash
0,b160c319,d2786168,w21z6
1,9c8f010e,d2786168,w21ze
2,03eb25e1,5055ab25,w21ze
3,3613129a,8984acb6,w23b1
4,0946c9e5,1e3a2913,w21z4


In [64]:
# Rename vendor geohash to vendor_geohash for clarity
geo_vendors = geo_vendors.rename(columns={'geohash': 'vendor_geohash'})

In [69]:
geo_vendors.head()

,vendor_id,chain_id,vendor_geohash
0,b160c319,d2786168,w21z6
1,9c8f010e,d2786168,w21ze
2,03eb25e1,5055ab25,w21ze
3,3613129a,8984acb6,w23b1
4,0946c9e5,1e3a2913,w21z4


In [76]:
geo_vendors.isna().sum()

vendor_id            0
chain_id          1499
vendor_geohash       0
dtype: int64

## 1.2 Geohash Exploration

In [71]:
# Merge orders with vendors to get both customer and vendor geohashes
merged = pd.merge(orders, vendors, on='vendor_id', how='left')

merged_geo = geo_df.merge(geo_vendors, on='vendor_id', how='left')

merged_geo.head()

,customer_id,customer_geohash,order_id,vendor_id,chain_id,vendor_geohash
0,1ba124d4e5,w21z7,0,212753d2,66c9978d,w21z7
1,1ba124d4e5,w21z7,0,212753d2,66c9978d,w21z7
2,1ba124d4e5,w21z7,0,212753d2,66c9978d,w21z7
3,1ba124d4e5,w21z7,0,212753d2,66c9978d,w21z7
4,1ba124d4e5,w21z7,0,212753d2,66c9978d,w21z7


In [ ]:
print(f"Total orders: {geo_df['order_id'].nunique()}")
print(f"Unique customers: {geo_df['customer_id'].nunique()}")
print(f"Unique vendors: {geo_df['vendor_id'].nunique()}")
print(f"Unique chains: {geo_df['chain_id'].nunique()}")

Total orders: 12181
Unique customers: 2650
Unique vendors: 3905
Unique chains: 1166


In [75]:

print("Unique customer geohashes:", merged_geo['customer_geohash'].nunique())
print("Unique vendor geohashes:", merged_geo['vendor_geohash'].nunique())
print("\nTop customer cells:\n", merged_geo['customer_geohash'].value_counts().head())
print("\nTop vendor cells:\n", merged_geo['vendor_geohash'].value_counts().head())


Unique customer geohashes: 32
Unique vendor geohashes: 29

Top customer cells:
 customer_geohash
w21z6    2672
w21ze    2257
w21zu    2134
w23b4    2044
w21zt    1817
Name: count, dtype: int64

Top vendor cells:
 vendor_geohash
w21z7    2523
w23b4    2265
w21ze    2105
w21z6    2084
w21zt    1996
Name: count, dtype: int64


In [ ]:
geo_df['same_geohash_area'] = geo_df['customer_geohash'].str[:4] == geo_df['vendor_geohash'].str[:4]
geo_df['geohash_match_length'] = geo_df.apply(
    lambda row: sum(1 for a, b in zip(row['customer_geohash'], row['vendor_geohash']) if a == b),
    axis=1
)

# This tells you: Can this vendor deliver to this customer's area?

/tmp/ipykernel_1982/1800689423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df['same_geohash_area'] = geo_df['customer_geohash'].str[:4] == geo_df['vendor_geohash'].str[:4]
/tmp/ipykernel_1982/1800689423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df['geohash_match_length'] = geo_df.apply(


In [ ]:
# Create user-item interaction matrix
interaction_matrix = geo_df.groupby(['customer_id', 'vendor_id']).size().unstack(fill_value=0)

# This becomes the foundation for "users who ordered from X also ordered from Y"

In [ ]:
# Analyze if customers prefer specific chains
chain_preferences = geo_df.groupby(['customer_id', 'chain_id']).size().unstack(fill_value=0)

# This helps with: "Because you like this chain, try these other locations"

In [ ]:
pairs = (geo_df
         .groupby(['vendor_geohash','customer_geohash'])
         .size().reset_index(name='count'))

m2 = folium.Map(location=[center_lat, center_lon], zoom_start=12)

for _, r in pairs.iterrows():
    vlat, vlon = pgh.decode(r['vendor_geohash'])
    clat, clon = pgh.decode(r['customer_geohash'])
    folium.PolyLine([(vlat, vlon), (clat, clon)], weight=1, opacity=min(0.9 + r['count']/pairs['count'].max(), 0.9)).add_to(m2)

m2

# **2. Geohash x Products** 

In [52]:
orders['order_day'].unique()

array(['85 days', '61 days', '8 days', '59 days', '81 days', '29 days',
       '87 days', '70 days', '55 days', '75 days', '66 days', '89 days',
       '47 days', '34 days', '18 days', '54 days', '30 days', '38 days',
       '76 days', '42 days', '56 days', '41 days', '48 days', '35 days',
       '7 days', '19 days', '21 days', '49 days', '46 days', '12 days',
       '11 days', '36 days', '77 days', '33 days', '13 days', '53 days',
       '39 days', '16 days', '45 days', '65 days', '4 days', '25 days',
       '67 days', '22 days', '32 days', '60 days', '37 days', '83 days',
       '27 days', '6 days', '2 days', '5 days', '1 days', '88 days',
       '51 days', '20 days', '71 days', '58 days', '72 days', '86 days',
       '44 days', '17 days', '79 days', '14 days', '68 days', '31 days',
       '63 days', '64 days', '24 days', '26 days', '0 days', '3 days',
       '43 days', '50 days', '52 days', '15 days', '40 days', '23 days',
       '28 days', '69 days', '80 days', '78 days', '57 days'

# **3. Geohash x Vendors** 